In [91]:
import os, re, sys, pandas as pd
TXT_DIR = '/Users/heejinohn/trantxt/txt'
filename ='AIV_20170428.pdf.txt'

In [92]:
# with open(r'/Users/heejinohn/trantxt/master_csv.csv', 'r') as csvfile:
#     for linenum, line in enumerate(csvfile):
#         splitted_line = line.split(",")
#         filename = splitted_line[1].rstrip()
#         cusip = splitted_line[0].rstrip()
#         print(linenum, filename, " started.")
#####################################################################################
# read in file
#####################################################################################
file = open(os.path.join(TXT_DIR, filename),'r')
text = file.read()
file.close()
#####################################################################################
# split lines 
#####################################################################################
lines = [line.strip() for line in text.split('\n')]
lines = [' '.join(x.split()) for x in lines if x=='' or len(x) >= 3]
if next(s for s in lines if s) == 'Corrected Transcript':
    doc_error = 0
    doc = next((x for x in lines if x.endswith('Transcript')), None)
    pat1 = re.compile(r"^(([0-9])|([0-2][0-9])|([3][0-1]))[- /.](Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[- /.](19|20)\d\d$")
    date = next((x for x in lines if pat1.match(x)), None)
    pat2 = re.compile(r"^(?:.*)\s\((?:[A-Z]+(?:\.XX(?:[0-9]|1[0-9])|\.[A-Z]+(?:\.[A-Z]+)?|\s?Q[0-5])?|[0-9A-Z]+\-[A-Z])\)$")
    comp = next((x for x in lines if pat2.match(x)), None)
    pat4 = re.compile(r"^((March|September|December|FY|Q[1-5])\s+\d{4}\s+|Business Update)")
    evnt = next((x for x in lines if pat4.match(x)), None)
    if comp is not None:
        tic = comp[comp.find("(")+1:comp.find(")")]
    else:
        tic = ""
#####################################################################################
# write doc file for documents other than corrected transcripts
#####################################################################################
# else:
#     doc_error = 1
#     doc = pd.DataFrame(data = [(cusip, 1, "", "")],index=[filename])
#     doc.to_csv(r'/Users/heejinohn/trantxt/doc.csv', mode='a', header=False, sep='|')
#     print('\t', linenum, filename, ' completed.')
# All subsequent codes apply only to documents identified as corrected transcripts.


if doc_error == 0:
#####################################################################################
# remove clutters
#####################################################################################
    lines = [line for line in lines if line[:5]!="1-877" \
             and line.split(' ', 1)[0]!='Copyright' \
             and line.split(':', 1)[0]!='Total Pages' \
             and line != date \
             and "".join(line.split()) != "".join(comp.split()) \
             and line != evnt \
             and line != doc \
             and line != "("+tic+")"
            ]
    discl = len(lines) - 1 - lines[::-1].index('Disclaimer')
    lines = lines[:discl]
    from itertools import groupby
    lines = [x[0] for x in groupby(lines)]
    clean_txt = lines.copy()
#####################################################################################
# identify management:position
#####################################################################################
    # def chunks(l, n):
    #     chunk=[]
    #     """Yield successive n-sized chunks from l."""
    #     for i in range(0, len(l), n):
    #         chunk.append(l[i:i + n])
    #     return chunk

    if "CORPORATE PARTICIPANTS" in clean_txt:
        start = len(clean_txt) - 1 - clean_txt[::-1].index('CORPORATE PARTICIPANTS')
        end = next(i for i, x in enumerate(clean_txt) if x.startswith('......') or x == 'OTHER PARTICIPANTS')
        blanks = [i for i, x in enumerate(clean_txt[start:end]) if x=='']
        mgmt_names =[clean_txt[start:end][blanks[i]+1] for i in range(len(blanks)-1)]
        mgmt_pos = []
        for i in range(len(blanks)-1):
            if blanks[i+1] - blanks[i]==3:
                mgmt_pos.append(clean_txt[start:end][blanks[i]+2])
            else:
                pos = " ".join(clean_txt[start:end][blanks[i]+2:blanks[i+1]])
                mgmt_pos.append(pos)
        mgmt_error = 0
        del clean_txt[:end + 1]
#         mgmt = pd.DataFrame(data = list(zip(mgmt_names,mgmt_pos)), index = [[filename] * len(mgmt_names), list(range(len(mgmt_names)))])
#         mgmt.to_csv(r'/Users/heejinohn/trantxt/mgmt.csv', mode='a', header=False, sep='|')
    else:
        mgmt_error = 1
#         mgmt = pd.DataFrame(data = [("", 1, "", "")],index=[filename])
#         mgmt.to_csv(r'/Users/heejinohn/trantxt/mgmt.csv', mode='a', header=False, sep='|')



In [93]:
#####################################################################################
# identify analyst:firm
#####################################################################################
if "OTHER PARTICIPANTS" in clean_txt:
    start = len(clean_txt) - 1 - clean_txt[::-1].index('OTHER PARTICIPANTS')
    end = next(i for i, x in enumerate(clean_txt) if x.startswith('......') or x == 'MANAGEMENT DISCUSSION SECTION')
    blanks = [i for i, x in enumerate(clean_txt[start:end]) if x=='']
    analyst_names =[clean_txt[start:end][blanks[i]+1] for i in range(len(blanks)-1)]
    analyst_emps = []
    for i in range(len(blanks)-1):
        if blanks[i+1] - blanks[i]==3:
            analyst_emps.append(clean_txt[start:end][blanks[i]+2])
        else:
            pos = " ".join(clean_txt[start:end][blanks[i]+2:blanks[i+1]])
            analyst_emps.append(pos)
    analyst_error = 0
    del clean_txt[:end + 1]
#         analyst = pd.DataFrame(data = list(zip(analyst_names,analyst_emps)), index = [[filename] * len(analyst_names), list(range(len(analyst_names)))])
#         analyst.to_csv(r'/Users/heejinohn/trantxt/analysts.csv', mode='a', header=False, sep='|')
else:
    analyst_error = 1
    analyst_names = []
#         analyst = pd.DataFrame(data = [("", 1, "", "")],index=[filename])
#         analyst.to_csv(r'/Users/heejinohn/trantxt/analysts.csv', mode='a', header=False, sep='|')
#####################################################################################
# remove operator dialogues
#####################################################################################
clean_txt = [i for i in clean_txt if i != ""]
operators = [i for i, x in enumerate(clean_txt) if x[:9] == "Operator:"]
splits = [i for i, x in enumerate(clean_txt) if x.startswith('......') or x in analyst_names + mgmt_names]
drops = [[x,next((i for i in splits if i > x), None)] for x in operators]
for x in drops[::-1]:
    if x[1] == None:
        del clean_txt[x[0]:]
    else:
        if x[1]-x[0]==1:
            del clean_txt[x[0]]
        else:
            del clean_txt[x[0]:x[1]]

In [94]:
#####################################################################################
# divide management discussion and question and answer sections
#####################################################################################
if 'QUESTION AND ANSWER SECTION' in clean_txt:
    sec = clean_txt.index('QUESTION AND ANSWER SECTION')
    mds = clean_txt[:sec]
    qnas = clean_txt[sec + 1:]
    qnas_error = 0
else:
    mds = clean_txt
    qna_clean_txt = ""
    qnas_error = 1
#####################################################################################
# clean management discussion section
#####################################################################################
mds_clean = [x for x in mds if x not in mgmt_names \
            and x not in mgmt_pos \
            and not x.startswith('......') \
            and (x != x.upper() or not re.search('[a-zA-Z]', x))]
mds_clean_txt = " ".join(mds_clean)
#####################################################################################
# write doc file for corrected transcripts
#####################################################################################
#     doc = pd.DataFrame(data = [(cusip, doc_error, qnas_error, mds_clean_txt)],index=[filename])
#     doc.to_csv(r'/Users/heejinohn/trantxt/doc.csv', mode='a', header=False, sep='|')
#####################################################################################
# clean question and answer section
#####################################################################################
if mgmt_error == 0 and analyst_error == 0 and qnas_error == 0:
    qnas_clean = [x for x in qnas if x.startswith('.......') \
                  or x.upper() != x or not re.search('[a-zA-Z]', x)]
    qnas_clean = [x for x in qnas_clean if x not in mgmt_pos and x not in analyst_emps]
    for n, i in enumerate(qnas_clean):
        if i.startswith('<'):
            if ':' in i:
                qnas_clean.insert(n+1, i.split(':',1)[1])
                split_i = i.split(':',1)[0]
            elif ']' in i:
                qnas_clean.insert(n+1, i.split(']',1)[1])
                split_i = i.split(']',1)[0]
            else:
                split_i = i
            capped = set(re.findall(r'(\b[A-Z][a-zA-Z\'\-]+\b)', split_i))
            if not capped:
                capped = set(re.findall(r'(\b[A-Z][a-zA-Z\'\-]+\b)', i))
            a_last = [i[-1] for i in [i.split() for i in analyst_names]]
            a_first = [j for fullname in analyst_names for j in fullname.split()[:-1]]
            m_last = [i[-1] for i in [i.split() for i in mgmt_names]]
            m_first = [j for fullname in mgmt_names for j in fullname.split()[:-1]]
            if capped&set(a_last) and not capped&set(m_last):
                if len(capped&set(a_last)) == 1:
                    qnas_clean[n] = analyst_names[a_last.index(next(iter(capped&set(a_last))))]
                else:
                    qnas_clean[n] = analyst_names[a_first.index(next(iter(capped&set(a_first))))]
            elif capped&set(m_last) and not capped&set(a_last):
                if len(capped&set(m_last)) == 1:
                    qnas_clean[n] = mgmt_names[m_last.index(next(iter(capped&set(m_last))))]
                else:
                    qnas_clean[n] = mgmt_names[m_first.index(next(iter(capped&set(m_first))))]
            else:
                if capped&set(a_first):
                    qnas_clean[n] = analyst_names[a_last.index(next(iter(capped&set(a_last))))]
                elif capped&set(m_first):
                    qnas_clean[n] = mgmt_names[m_last.index(next(iter(capped&set(m_last))))]

    splits = [i for i, x in enumerate(qnas_clean) if x.startswith('......')]
    mgmt_locs = [i for i, x in enumerate(qnas_clean) if x in mgmt_names]
    analyst_locs = [i for i, x in enumerate(qnas_clean) if x in analyst_names]
    sides = []
    for split in splits:
        if split == 0:
            sides.append(split + 1)
        else:
            sides.extend([split - 1, split + 1])
    add_splits = [x for x in list(set(mgmt_locs)-set(sides))+list(set(analyst_locs)-set(sides))]
    combines = sorted(splits + add_splits)
    qna_clean_txt = []
    for i in range(len(combines)-1):
        if combines[i] in splits and combines[i+1]-combines[i] > 1:
            qna_clean_txt.append(qnas_clean[combines[i]+1])
            qna_clean_txt.append(" ".join(qnas_clean[combines[i]+2:combines[i+1]]))
        elif combines[i] in add_splits and combines[i+1]-combines[i] > 1:
            qna_clean_txt.append(qnas_clean[combines[i]])
            qna_clean_txt.append(" ".join(qnas_clean[combines[i]+1:combines[i+1]]))
    if combines[-1] != len(qnas_clean):
        if combines[-1] in splits and len(qnas_clean)-combines[-1] > 1:
            qna_clean_txt.append(qnas_clean[combines[-1]+1])
            qna_clean_txt.append(" ".join(qnas_clean[combines[-1]+2:]))
        elif combines[-1] in add_splits and len(qnas_clean)-combines[-1] > 1:
            qna_clean_txt.append(qnas_clean[combines[-1]])
            qna_clean_txt.append(" ".join(qnas_clean[combines[-1]+1:]))
    an_emp = dict(zip(analyst_names, analyst_emps))
    an_loc = {analyst:qna_clean_txt.index(analyst) for analyst in analyst_names if analyst in qna_clean_txt}
    analyst_names = sorted(analyst_names, key = lambda x: (x not in an_loc, an_loc.get(x, None)))
    analyst_emp = [an_emp[name] for name in analyst_names]
    q = dict()
    l = dict()

    for i, name in enumerate(analyst_names):
        q[i] = [qna_clean_txt[j+1] for j, line in enumerate(qna_clean_txt) if line == name]
        l[i] = [j+1 for j, line in enumerate(qna_clean_txt) if line == name]
    a = {k: [] for k in list(l.keys())}

    for key in l.keys():
        if len(l[key]) > 0:
            others = [name for i, name in enumerate(analyst_names) if i != key]
            for n, loc in enumerate(l[key][:-1]):
                end = min(l[key][n+1], next((i for i, line in enumerate(qna_clean_txt) if line in others and i > loc), len(qna_clean_txt)))
                if any(x in mgmt_names for x in qna_clean_txt[loc:end]):
                    a[key].append(" ".join([qna_clean_txt[loc:end][j+1] for j,line in enumerate(qna_clean_txt[loc:end]) if line in mgmt_names]))
                else:
                    a[key].append("")
            if l[key][-1] == max(value for values in l.values() for value in values):
                if len([x for x in q[key][-1].split(". ") if not re.search(r'[T|t]hanks?(?:\syou)?|appreciate|[O|o]kay|[G|g]ood|[G|g]reat|[Y|y]eah', x)]) < 3:
                    a[key].append("")
                else:
                    if any(x in mgmt_names for x in qna_clean_txt[l[key][-1]:]):
                        a[key].append(" ".join([qna_clean_txt[l[key][-1]:][j+1] for j,line in enumerate(qna_clean_txt[l[key][-1]:]) if line in mgmt_names]))
                    else:
                        a[key].append("")
            else:
                end = next(i for i, line in enumerate(qna_clean_txt) if line in others and i > l[key][-1])
                if any(x in mgmt_names for x in qna_clean_txt[l[key][-1]:end]):
                    a[key].append(" ".join([qna_clean_txt[l[key][-1]:end][j+1] for j,line in enumerate(qna_clean_txt[l[key][-1]:end]) if line in mgmt_names]))
                else:
                    a[key].append("")
#####################################################################################
# write qna file
#####################################################################################                            
#         index = pd.MultiIndex.from_tuples([(filename, key, i) for key in q for i in range(len(q[key]))])
#         qa = pd.DataFrame(index = index, columns =['q', 'a'])
#         for key in q:
#             if len(q[key]) > 0:
#                 for i in range(len(q[key])):
#                     qa.at[(filename, key, i), "q"] = q[key][i]
#                     qa.at[(filename, key, i), "a"] = a[key][i]
#         qa.to_csv(r'/Users/heejinohn/trantxt/qna.csv', mode='a', header=False, sep='|')
#####################################################################################
# clean slate
#####################################################################################                                            
#     for name in dir():
#         if name in ['doc', 'mgmt', 'analyst', 'qa']:
#             del globals() [name]
#     print('\t', linenum, filename, ' completed.')

In [95]:
analyst = pd.DataFrame(data = list(zip(analyst_names,analyst_emps)), index = [[filename] * len(analyst_names), list(range(len(analyst_names)))])
analyst.to_csv(r'/Users/heejinohn/trantxt/analysts_error.csv', mode='a', header=False, sep='|')

In [96]:
index = pd.MultiIndex.from_tuples([(filename, key, i) for key in q for i in range(len(q[key]))])
qa = pd.DataFrame(index = index, columns =['q', 'a'])
for key in q:
    if len(q[key]) > 0:
        for i in range(len(q[key])):
            qa.at[(filename, key, i), "q"] = q[key][i]
            qa.at[(filename, key, i), "a"] = a[key][i]
qa.to_csv(r'/Users/heejinohn/trantxt/qna_error.csv', mode='a', header=False, sep='|')

